In [ ]:
#r "D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Foundation.XDG;
using BoSSS.Solution;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

databases


Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


Databases loaded: 


Capacity: 4
Count: 3



#0: { Session Count = 16; Grid Count = 24; Path = D:\local\local_test_db2 }
#1: { Session Count = 91; Grid Count = 112; Path = \\hpccluster\hpccluster-scratch\smuda\XNSE_testDB }
#2: { Session Count = 819; Grid Count = 23491; Path = \\HPCCLUSTER\hpccluster-scratch\smuda\XNSE_studyDB }


In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,"[ { D:\local\ == : LocalMountPath: D:\local\, PathAtRemote: } ]",,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,"[ { \\hpccluster\hpccluster-scratch\smuda\ == : LocalMountPath: \\hpccluster\hpccluster-scratch\smuda\, PathAtRemote: } ]",FDY\smuda,<null>,DC2,<null>,Normal,True


In [ ]:
var myBatch = ExecutionQueues[1];
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,Password,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,FDY\smuda,<null>,DC2,<null>,Normal,True,"[ { \\hpccluster\hpccluster-scratch\smuda\ == : LocalMountPath: \\hpccluster\hpccluster-scratch\smuda\, PathAtRemote: } ]"


In [ ]:
IDatabaseInfo myDb = myBatch.CreateOrOpenCompatibleDatabase("XNSE_testDB");
myDb

{ Session Count = 91; Grid Count = 112; Path = \\hpccluster\hpccluster-scratch\smuda\XNSE_testDB }

In [ ]:
//BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.Init("Droplet3D_LegendreSetup_paperStudy"); 

Project name is set to 'Droplet3D_LegendreSetup_paperStudy'.


In [ ]:
// ===============
// Test case setup
// ===============
double R0 = 0.001;
double L  = 3.0 * R0;

int m = 2;    // Legendre polynom
//double[] T = new double[] {5e-3, 2.5e-3, 1.7e-3};    // period length corresponding to mode m  
double[] aP = new double[] {0.5, 0.7, 0.9};    //  amplitude of the initial deformation
int aP_Index = 0;

In [ ]:
// ================
// physical setting
// ================
double ratio = 0.001;
int setup    = 3;

string setupTag;
string OhTag;
double rhoA = 1260;
double rhoB = rhoA * ratio;
double muA;
double muB;
double sigma = 0.007;

switch (setup) {
case 1: {
setupTag = "Ohnesorge Zahl = 0.76";    // critical Oh for m = 2
OhTag    = "076";
muA      = 0.0714;
muB      = muA* ratio;
break;
}
case 2: {
setupTag = "Ohnesorge Zahl = 0.55";    // critical Oh for m = 4
OhTag    = "055";
muA      = 0.0517;
muB      = muA* ratio;
break;
}
case 3: {
setupTag = "Ohnesorge Zahl = 0.1";
OhTag    = "01";
muA      = 0.0094;
muB      = muA* ratio;
break;
}
default:
break;
}

In [ ]:
// ===========================
// Grids for Convergence Study
// ===========================
var GridSeq = new IGridInfo[1];
int[] kelemSeq = new int[]{6};
bool quarterDomain = true;

In [ ]:
for(int iGrid = 0; iGrid < GridSeq.Length; iGrid++) {
    
    int kelem = kelemSeq[iGrid];
    
    GridCommons grd;
    
if (!quarterDomain) {
    
    double[] xNodes = GenericBlas.Linspace(-L, L, kelem + 1);
    double[] yNodes = GenericBlas.Linspace(-L, L, kelem + 1);  
    double[] zNodes = GenericBlas.Linspace(-L, L, kelem + 1);    
    grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);

    grd.EdgeTagNames.Add(1, "wall");
 
    grd.DefineEdgeTags(delegate (double[] X) {
        byte et = 0;
            if (Math.Abs(X[1] + L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[1] - L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[0] + L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[0] - L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[2] + L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[2] - L) <= 1.0e-8)
                et = 1;
            return et;
    });
            
    grd.Name = "Droplet3D_test"; 
    
} else {

    double[] xNodes = GenericBlas.Linspace(0, L, kelem + 1);
    double[] yNodes = GenericBlas.Linspace(0, L, kelem + 1);  
    double[] zNodes = GenericBlas.Linspace(-L, L, (2*kelem) + 1);    
    grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);

    grd.EdgeTagNames.Add(1, "wall");
    grd.EdgeTagNames.Add(2, "slipsymmetry");
 
    grd.DefineEdgeTags(delegate (double[] X) {
        byte et = 0;
            if (Math.Abs(X[1] + 0) <= 1.0e-8)
                et = 2;
            if (Math.Abs(X[1] - L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[0] + 0) <= 1.0e-8)
                et = 2;
            if (Math.Abs(X[0] - L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[2] + L) <= 1.0e-8)
                et = 1;
            if (Math.Abs(X[2] - L) <= 1.0e-8)
                et = 1;
            return et;
    });
            
    grd.Name = "Droplet3D_quarterDomain"; 
    
}   

    //myDb.SaveGrid(ref grd);
    var igrd = IDatabaseInfoExtensions.SaveGrid(myDb, grd);
    
    GridSeq[iGrid] = igrd;
}

Grid Edge Tags changed.


An equivalent grid is already present in the database -- the grid will not be saved.


In [ ]:
// ======================
// Create Control objects
// ======================

int[] degS = new int[] { 3 };
int NoProc = 3;

List<XNSE_Control> controls = new List<XNSE_Control>();
//List<int> procPerCntrl = new List<int>();

In [ ]:
for(int iDeg = 0; iDeg < degS.Length; iDeg++) {
    for(int iGrd = 0; iGrd < GridSeq.Length; iGrd++) {
        
        XNSE_Control C = BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.Droplet.OscillatingDroplet3D_LegendrePolynomials(R0, m, aP_Index);
        
        C.CutCellQuadratureType = XQuadFactoryHelper.MomentFittingVariants.Saye;
        
        C.savetodb          = true;
        C.SetDatabase(myDb);
        C.ContinueOnIoError = false;
    
        int pDeg = degS[iDeg];   
        var grd  = GridSeq[iGrd];
    
        C.SetDGdegree(pDeg);
        
        C.SetGrid(grd);
        C.AddBoundaryValue("wall");
        C.AddBoundaryValue("slipsymmetry");
    
        C.Tags.Add(setupTag);
        C.PhysicalParameters.rho_A = rhoA;
        C.PhysicalParameters.rho_B = rhoB;
        C.PhysicalParameters.mu_A  = muA;
        C.PhysicalParameters.mu_B  = muB;
        C.PhysicalParameters.Sigma = sigma;
        
        C.PhysicalParameters.IncludeConvection = true;
    
        
        C.dtMin         = 5e-5;
        C.dtMax         = 5e-5;
        C.Endtime       = 1.0; //4.0 * T[0];
        C.NoOfTimesteps = 1000; //(int)((4.0 * T[0]) / 1e-3);
        
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(new BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater.AMRonNarrowband() { maxRefinementLevel = 1 });
        C.AMR_startUpSweeps = 1;
        
        //C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.FastMarching;
        //C.FastMarchingPenaltyTerms = BoSSS.Solution.LevelSetTools.Smoothing.JumpPenalization.jumpPenalizationTerms.Jump;
        //C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;  
        
        C.saveperiod = 2;
        
        C.SessionName = "Droplet3D_k" + pDeg + "_mesh6_LPm" + m + "_Oh" + OhTag + "AMR1";
        
        C.LinearSolver = BoSSS.Solution.Control.LinearSolverCode.direct_pardiso.GetConfig();
        //C.LinearSolver          = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
        //C.LinearSolver.NoOfMultigridLevels = 3;
        C.LinearSolver.ConvergenceCriterion   = 10e-8;
        C.NonLinearSolver.MinSolverIterations = 3;
        
        //controls[iDeg,iGrd] = C;
        controls.Add(C);
        //C.CreateJob(); // job will be permanently linked to control object
    
    }
    }

In [ ]:
controls.Count

1

In [ ]:
controls.ElementAt(0).SessionName

Droplet3D_k3_mesh6_LPm2_Oh01AMR1

In [ ]:
foreach(var cntrl in controls) {
    var job = new Job(cntrl.SessionName, typeof(XNSE));
    job.SetControlObject(cntrl);
    //job.NumberOfMPIProcs = NoProc;
    Console.WriteLine("Submitting Session: {0}", cntrl.SessionName);
    job.Activate(myBatch, false);
}

Submitting Session: Droplet3D_k3_mesh6_LPm2_Oh01AMR1


Loading session d5d0e3a2-b47e-4602-b644-7be3d6a02bc6 failed with message 'Unexpected end when deserializing object. Path 'm_KeysAndQueries.PressureBlockPrecondMode'.'


Loading session b3e86543-1b07-405c-a359-ae9592e5d3df failed with message 'Unexpected end when deserializing object. Path 'm_KeysAndQueries.PressureBlockPrecondMode'.'


Loading session 097e1b7d-e16c-4525-a5dc-1c47cd35a00a failed with message 'Unexpected end when deserializing object. Path 'm_KeysAndQueries.Timestepper_Scheme'.'


Loading session 137b157c-93c2-4d88-9971-8d3c688eba75 failed with message 'Could not find file '\\HPCCLUSTER\hpccluster-scratch\smuda\XNSE_studyDB\sessions\137b157c-93c2-4d88-9971-8d3c688eba75\Session.info'.'


...

Loading session b3e86543-1b07-405c-a359-ae9592e5d3df failed with message 'Unexpected end when deserializing object. Path 'm_KeysAndQueries.PressureBlockPrecondMode'.'


Loading session 097e1b7d-e16c-4525-a5dc-1c47cd35a00a failed with message 'Unexpected end when deserializing object. Path 'm_KeysAndQueries.Timestepper_Scheme'.'


Loading session 137b157c-93c2-4d88-9971-8d3c688eba75 failed with message 'Could not find file '\\HPCCLUSTER\hpccluster-scratch\smuda\XNSE_studyDB\sessions\137b157c-93c2-4d88-9971-8d3c688eba75\Session.info'.'


Loading session 28adc7a7-a336-435b-b560-d3c02f46c43d failed with message 'Could not find file '\\HPCCLUSTER\hpccluster-scratch\smuda\XNSE_studyDB\sessions\28adc7a7-a336-435b-b560-d3c02f46c43d\Session.info'.'


Loading session b2086d2b-23ca-46e4-b4a9-fb1a580007cb failed with message 'Could not find file '\\HPCCLUSTER\hpccluster-scratch\smuda\XNSE_studyDB\sessions\b2086d2b-23ca-46e4-b4a9-fb1a580007cb\Session.info'.'


In [ ]:
//BoSSSshell.WorkflowMgm.Sessions